In [ ]:
{
    "demand_nodes": {
        "filename": "data/max_100.csv",
        "x_column": "X",
        "y_column": "Y", 
        "budget_column": "mvmax"
    },
    "network_algorithm": "mod_boruvka",
    "network_parameters": {
        "minimum_node_count": 2
    }
}

In [ ]:
# python scripts/run_networker.py test/networker_config_max100.json -o output

import argparse
import json
import sys
import os
import networker
import logging
from networker import networker_runner

# setup log
logger = logging.getLogger('networker')

logger.info("networker %s (Python %s)" % (
                networker.__version__,
                '.'.join(map(str, sys.version_info[:3]))))

parser = argparse.ArgumentParser(description="Run networker")
parser.add_argument("config_filename",
                    help="json config file for running networker")
parser.add_argument("--working_directory", "-w", \
        default=".", \
        help="base directory which all input paths are relative to")
parser.add_argument("--output_directory", "-o", \
        default=".", \
        help="directory where all output files will be written")

args = parser.parse_args()

In [ ]:
# switch working dir BEFORE loading config
os.chdir(args.working_directory)
cfg = json.load(open(args.config_filename))
nwk = networker_runner.NetworkerRunner(cfg, args.output_directory)

nwk.validate()
nwk.run()


from sequencer import NetworkPlan
from sequencer.Models import EnergyMaximizeReturn

from os.path import join
source_folder = 'data/sumaila/input'

csv = join(source_folder, 'metrics-local.csv')
shp = join(source_folder, 'networks-proposed.shp')

nwp = NetworkPlan(shp, csv, prioritize='Population')
model = EnergyMaximizeReturn(nwp)

results = model.sequence()
target_folder = 'xyz'

from os import makedirs
try:
    makedirs(target_folder)
except OSError:
    pass

model.output(target_folder)